In [41]:
# Cell 1: Import necessary libraries

import heapq
import copy
import time


In [42]:
class Node:
    def __init__(self, bays, crane_position, crane_container_held, cost, actions):
        self.bays = bays
        self.crane_position = crane_position
        self.crane_container_held = crane_container_held
        self.cost = cost
        self.actions = actions

    def __lt__(self, other):
        return self.cost < other.cost

    def print_state(self):
        """ Prints the current state visually """
        bay_names = ["A", "B", "C", "D"]
        for i in range(len(self.bays)):
            bay_display = ", ".join(map(str, self.bays[i])) if self.bays[i] else ""
            bay_label = bay_names[i]
            if i == self.crane_position:
                print(f"{bay_label}: [{bay_display}] <-[ Holding: {self.crane_container_held} ]" if self.crane_container_held else f"[{bay_display}] <-[Empty]")
            else:
                print(f"[{bay_display}]")
        print(f"Total cost: {self.cost}\n")


In [43]:
def is_valid_drop(bays, crane_position, crane_container_held, stacking_limits):
    """ Checks if the drop action is valid """
    if crane_container_held is None:
        return False
    if len(bays[crane_position]) >= stacking_limits[crane_position]:  # Check stacking limit
        return False
    if bays[crane_position] and crane_container_held > bays[crane_position][-1]:  # Heavy over light
        return False
    return True

def is_valid_pick(bays, crane_position, crane_container_held):
    """ Checks if the pick action is valid """
    if crane_container_held is not None:
        return False
    if not bays[crane_position]:  # No container to pick
        return False
    return True

In [44]:
def perform_action(state, action, stacking_limits):
    """ Performs an action and returns a new state """
    new_bays = copy.deepcopy(state.bays)
    new_position = state.crane_position
    new_held = state.crane_container_held
    new_cost = state.cost + 1
    new_actions = state.actions + [action]

    if action == "RIGHT":
        if new_position < len(new_bays) - 1:
            new_position += 1
            if new_held is not None and new_held > 4: # condition for heavy containers
                new_cost += 1 # Additional time for moving with a heavy container
        else:
            return None

    elif action == "LEFT":
        if new_position > 0:
            new_position -= 1
            if new_held is not None and new_held > 4:
                new_cost += 1
        else:
            return None

    elif action == "PICK":
        if is_valid_pick(new_bays, new_position, new_held):
            pick_cost = 5 - len(new_bays[new_position])
            new_held = new_bays[new_position].pop()
            new_cost += pick_cost  # Picking cost
        else:
            return None

    elif action == "DROP":
        if is_valid_drop(new_bays, new_position, new_held, stacking_limits):
            new_bays[new_position].append(new_held)
            new_held = None
            new_cost += 1  # Dropping cost
        else:
            return None

    return Node(new_bays, new_position, new_held, new_cost, new_actions)


In [45]:
def heuristic(state, goal_bays):
    """ Calculates a heuristic estimate for A* """
    misplaced = 0
    for i, bay in enumerate(state.bays):
        if i >= len(goal_bays): continue
        for j, container in enumerate(bay):
            if j >= len(goal_bays[i]) or container != goal_bays[i][j]:
                misplaced += 1
    return misplaced + (1 if state.crane_container_held else 0)

In [46]:
def is_goal_state(state, goal_bays):
    """ Checks if the current state matches the goal state """
    return state.bays == goal_bays

In [55]:
def a_star_search(initial_state, goal_bays, stacking_limits):
    """ A* search algorithm """
    frontier = []
    heapq.heappush(frontier, (heuristic(initial_state, goal_bays), initial_state))
    visited = set()

    while frontier:
        _, current_state = heapq.heappop(frontier)

        if is_goal_state(current_state, goal_bays):
            return current_state.actions, current_state  # Return action sequence and final state

        state_tuple = (tuple(tuple(bay) for bay in current_state.bays), current_state.crane_position, current_state.crane_container_held)
        if state_tuple in visited:
            continue
        visited.add(state_tuple)

        for action in ["RIGHT", "LEFT", "PICK", "DROP"]:
            new_state = perform_action(current_state, action, stacking_limits)
            if new_state:
                f_cost = new_state.cost + heuristic(new_state, goal_bays)
                heapq.heappush(frontier, (f_cost, new_state))

    return None, None  # No solution found


In [56]:
initial_bays = [[1, 2, 3, 4, 5, 6], [], [], []]  # Some containers are present at the start
goal_bays = [[1, 2], [3], [4], [6, 5]]  # Goal state
stacking_limits = [2, 3, 2, 3]  # Limits per bay
initial_crane_position = 0  # Start position of crane
initial_crane_held = None  # Crane starts empty

initial_state = Node(initial_bays, initial_crane_position, initial_crane_held, 0, [])
solution, final_state = a_star_search(initial_state, goal_bays, stacking_limits)

if solution:
    print("Solution found:")
    current_state = initial_state
    for action in solution:
        current_state = perform_action(current_state, action, stacking_limits)
        if current_state:
            time.sleep(0.7)  # Small delay for visualization
            current_state.print_state()
else:
    print("No solution found")

Solution found:
A: [1, 2, 3, 4, 5] <-[ Holding: 6 ]
[]
[]
[]
Total cost: 0

[1, 2, 3, 4, 5]
B: [] <-[ Holding: 6 ]
[]
[]
Total cost: 2

[1, 2, 3, 4, 5]
[]
C: [] <-[ Holding: 6 ]
[]
Total cost: 4

[1, 2, 3, 4, 5]
[]
[]
D: [] <-[ Holding: 6 ]
Total cost: 6

[1, 2, 3, 4, 5]
[]
[]
[6] <-[Empty]
Total cost: 8

[1, 2, 3, 4, 5]
[]
[] <-[Empty]
[6]
Total cost: 9

[1, 2, 3, 4, 5]
[] <-[Empty]
[]
[6]
Total cost: 10

[1, 2, 3, 4, 5] <-[Empty]
[]
[]
[6]
Total cost: 11

A: [1, 2, 3, 4] <-[ Holding: 5 ]
[]
[]
[6]
Total cost: 12

[1, 2, 3, 4]
B: [] <-[ Holding: 5 ]
[]
[6]
Total cost: 14

[1, 2, 3, 4]
[]
C: [] <-[ Holding: 5 ]
[6]
Total cost: 16

[1, 2, 3, 4]
[]
[]
D: [6] <-[ Holding: 5 ]
Total cost: 18

[1, 2, 3, 4]
[]
[]
[6, 5] <-[Empty]
Total cost: 20

[1, 2, 3, 4]
[]
[] <-[Empty]
[6, 5]
Total cost: 21

[1, 2, 3, 4]
[] <-[Empty]
[]
[6, 5]
Total cost: 22

[1, 2, 3, 4] <-[Empty]
[]
[]
[6, 5]
Total cost: 23

A: [1, 2, 3] <-[ Holding: 4 ]
[]
[]
[6, 5]
Total cost: 25

[1, 2, 3]
B: [] <-[ Holding: 4 ]
[]